In [ ]:
library(parquetize)
library(aws.s3)
library(haven)

# Configuração do S3
Sys.setenv(
    "AWS_ACCESS_KEY_ID" = "<CHAVE-S3>",
    "AWS_SECRET_ACCESS_KEY" = "<SECRET-S3>",
    "AWS_S3_ENDPOINT"= "s3-h.ipe.df.gov.br"
)

OBJECT_OLD_KEY = 'DIEPS/bases/ped/NovaPEDDF2016.sav'
OBJECT_NEW_KEY = 'DIEPS/bases/ped-parquet/NovaPEDDF2016.parquet'
BUCKET_NAME = 'raw'
OUTPUT_NAME = '/tmp/output.parquet'

In [ ]:
# Verifica conexão com o MinIO
bucketlist(region = '', use_https = TRUE, verbose = TRUE)

In [ ]:


# Busca o objeto
tempfile = tempfile(fileext = '.sav')
object_path = paste0('s3://', BUCKET_NAME, '/', OBJECT_OLD_KEY)
filename = save_object(object = object_path, file = tempfile, region = '', use_https = TRUE)
filename

In [ ]:
data <- read_sav(filename)
head(data)

In [ ]:
summary(data)

# Conversão para parquet

Converte o dataframe para parquet

In [ ]:
table_to_parquet(
    path_to_file = filename,
    path_to_parquet = OUTPUT_NAME,
    encoding = 'utf-8',
    compression = 'snappy'
)


# Carregamento

Salva o arquivo de volta no bucket `raw`.

In [ ]:
put_object(
    bucket = BUCKET_NAME,
    object = OBJECT_NEW_KEY,
    file = OUTPUT_NAME,
    region = '',
    use_https = TRUE
)